## This process extracts the information about all IT stock markets from Yahoo Finance daily and then save them properly in a DB.

#### Global variables, constants and imports

In [1]:
from bs4 import BeautifulSoup
from pandas_datareader import data as pdr
from random import randint
from time import sleep
import json
import os
import pandas as pd
import quandl
import requests
import time
import fix_yahoo_finance as yf

PATH_PROJECT = os.getcwd()
queued_stocks = []

#### 1. Extraction of all Technology stocks (at 11th, Oct there are 754 stocks).

In [2]:
def technology_stocks():
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
    }
    try:
        base_url = 'https://api.iextrading.com/1.0/stock/market/collection/sector?collectionName=Technology'
        r = requests.get(base_url, headers = headers)
        if r.status_code == 200:
            src = BeautifulSoup(r.text, 'lxml')
            stock_list = json.loads(src.find('p').text)
            stocks = [stock["symbol"] for stock in stock_list]
            return stocks
        else:
            return None
    except Exception as ex:
        return None

#### 2. Using Quandl to obtain the daily data of each stock.

In [3]:
def daily_report():
    pass

#### 3. Using Quandl to obtain the daily data of each stock.

In [4]:
def historical_report(stock, start_date, end_date):
    yf.pdr_override()
    try:
        data = pdr.get_data_yahoo(
            stock, 
            start = start_date, 
            end = end_date, 
            progress = False
        )
        data['Stock'] = stock
        return data
    except Exception as ex:
        print('Yahoo Finance collapse downloading %s data. Adding to queue to continue once the correct downloads end...' % stock)
        queued_stocks.append(stock)

In [5]:
stocks = technology_stocks()
if stocks != None:
    start_date = '1955-01-01'
    end_date = '2018-07-31'
    historical = [historical_report(stock, start_date, end_date) for stock in stocks]
    historical_df = pd.concat(historical)
    historical_df['Open'] = pd.Series(['{:.15f}'.format(stock) for stock in historical_df['Open']], index = historical_df.index)
    historical_df['High'] = pd.Series(['{:.15f}'.format(stock) for stock in historical_df['High']], index = historical_df.index)
    historical_df['Low'] = pd.Series(['{:.15f}'.format(stock) for stock in historical_df['Low']], index = historical_df.index)
    historical_df['Close'] = pd.Series(['{:.15f}'.format(stock) for stock in historical_df['Close']], index = historical_df.index)
    historical_df['Adj Close'] = pd.Series(['{:.15f}'.format(stock) for stock in historical_df['Adj Close']], index = historical_df.index)
    historical_df['date_d'] = historical_df.index
    historical_df.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'stock', 'date_d']
    historical_df.to_csv(PATH_PROJECT + '/postgresql/stocks_historical_data.csv', index = False, sep = '\t', encoding = 'utf-8')

Yahoo Finance collapse downloading CBLK data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading DBX data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading GSKY data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading LEXEB data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading EVH data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading IIIV data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading LKM data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading BDC-B data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading PRSP data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downlo

In [6]:
historical_df
#historical_df['var3'] = pd.Series(['{:.15f}%'.format(stock) for stock in df['var3']], index = df.index)


,open,high,low,close,adj_close,volume,stock,date_d
Date,,,,,,,,
2014-04-04,40.000000000000000,40.799999000000000,34.000000000000000,34.000000000000000,34.000000000000000,10970100,GRUB,2014-04-04
2014-04-07,33.979999999999997,35.869999000000000,33.060001000000000,34.799999000000000,34.799999000000000,1152200,GRUB,2014-04-07
2014-04-08,35.250000000000000,36.619999000000000,34.599997999999999,35.990002000000004,35.990002000000004,510700,GRUB,2014-04-08
2014-04-09,36.099997999999999,36.500000000000000,34.610001000000004,35.310001000000000,35.310001000000000,455300,GRUB,2014-04-09
2014-04-10,35.369999000000000,35.369999000000000,30.815001000000002,30.910000000000000,30.910000000000000,754100,GRUB,2014-04-10
2014-04-11,30.780001000000002,34.470001000000003,29.860001000000000,33.639998999999996,33.639998999999996,1185600,GRUB,2014-04-11
2014-04-14,34.080002000000000,36.560001000000000,32.880001000000000,35.159999999999997,35.159999999999997,856200,GRUB,2014-04-14
2014-04-15,35.580002000000000,35.580002000000000,31.340000000000000,35.000000000000000,35.000000000000000,729600,GRUB,2014-04-15
2014-04-16,33.900002000000001,37.700001000000000,33.619999000000000,37.520000000000003,37.520000000000003,869600,GRUB,2014-04-16
